# DSBA Programming & Visualization
## Assignment 02


| **Item**            | **Details**              |
|----------------------|--------------------------|
| Deadline             | 26 oktober, 23:59u       |
| Data                 | olympics_raw.csv, flags_raw.csv, gdp_raw.csv, population_raw.csv  |
| Aantal vragen        | 4                        |


Lever een script of jupyter 
notebook in met daarin je code om de vragen te beantwoorden én in commentaren (#) 
uitleg waar nodig/gevraagd. Lever het script / notebook in via Canvas.

Gebruik vooral de slides en oefeningen die we in de colleges hebben behandeld

## Data
In deze assignment maken we wederom gebruik van een dataset over de Olympische
Spelen, 1896-2022. Daarnaast zijn gegevens beschikbaar over vlaggen,
inwonersaantallen en GDP. Het betreft de ruwe databestanden zoals die zijn gebruikt
om olympics_prepared.csv te maken, het bestand dat we in assignment 01 hebben
gebruikt. (Let op: olympics_prepared bevat alleen data van 1960–2022; vergelijkingen kunnen daardoor afwijken.)


Hieronder volgt een viertal vragen over deze dataset die met Python kunnen worden
beantwoord.

## Q1 – Transformations
Lees de data in vanuit de 4 `.csv`-bestanden.



In [99]:
# === Generic CSV loader: bestandsnaam -> DataFrame variabele ===
from pathlib import Path
import pandas as pd
import re, keyword
from typing import Dict

# ---- Instellingen
DATA_DIR = Path(".")          # of bijv. Path("./data")
PATTERN  = "*.csv"            # pas aan indien gewenst
NA_VALUES = ["", "NA", "N/A", "na", "null", "Null", "NULL"]

def to_var_name(p: Path) -> str:
    """
    Converteer bestandsnaam naar geldige Python variabelenaam.
    Voorbeeld: 'olympics_raw.csv' -> 'olympics_raw'
    """
    name = p.stem.lower()
    name = re.sub(r"[^0-9a-zA-Z]+", "_", name)       # niet-alfanumeriek -> _
    name = re.sub(r"_+", "_", name).strip("_")       # dubbele _ verwijderen
    if not name: 
        name = "df"
    if name[0].isdigit():
        name = f"df_{name}"
    if keyword.iskeyword(name):
        name = f"{name}_"
    return name

def read_csv_safely(path: Path) -> pd.DataFrame:
    """
    Leest CSV met een paar praktische defaults:
    - engine='python' + sep=None -> laat Pandas scheidingsteken 'sniffen'
    - encoding='utf-8-sig' verwerkt BOM correct
    - low_memory=False voor stabielere dtypes
    """
    try:
        return pd.read_csv(
            path,
            sep=None, engine="python",
            encoding="utf-8-sig",
            na_values=NA_VALUES,
            keep_default_na=True,
            low_memory=False
        )
    except Exception as e:
        # Fallback op klassieke komma-separatie
        return pd.read_csv(
            path,
            sep=",",
            encoding="utf-8-sig",
            na_values=NA_VALUES,
            keep_default_na=True,
            low_memory=False
        )

# ---- Inlezen
dfs: Dict[str, pd.DataFrame] = {}
csv_paths = sorted(DATA_DIR.glob(PATTERN))

for p in csv_paths:
    var = to_var_name(p)
    df  = read_csv_safely(p)
    dfs[var] = df
    globals()[var] = df   # maak ook een losse variabele aan
    print(f"Loaded: {var:<25} shape={df.shape}  from '{p.name}'")


Loaded: flags_raw                 shape=(193, 5)  from 'flags_raw.csv'
Loaded: gdp_raw                   shape=(266, 68)  from 'gdp_raw.csv'
Loaded: olympics_raw              shape=(20170, 6)  from 'olympics_raw.csv'
Loaded: population_raw            shape=(18944, 4)  from 'population_raw.csv'


In [100]:
# ---- Snelle check (toon de eerste 2 dataframes)
for i, (name, df) in enumerate(dfs.items()):
    if i >= 5: break
    display(df.head().style.set_caption(name))

,name,flag_nr_colors,flag_mainhue,flag_topleft_color,flag_botright_color
0,Afghanistan,5,green,black,green
1,Albania,3,red,red,red
2,Algeria,3,green,green,white
3,American-Samoa,5,blue,blue,red
4,Andorra,3,gold,blue,red


,country_name,country_code,indicator_name,indicator_code,x1960,x1961,x1962,x1963,x1964,x1965,x1966,x1967,x1968,x1969,x1970,x1971,x1972,x1973,x1974,x1975,x1976,x1977,x1978,x1979,x1980,x1981,x1982,x1983,x1984,x1985,x1986,x1987,x1988,x1989,x1990,x1991,x1992,x1993,x1994,x1995,x1996,x1997,x1998,x1999,x2000,x2001,x2002,x2003,x2004,x2005,x2006,x2007,x2008,x2009,x2010,x2011,x2012,x2013,x2014,x2015,x2016,x2017,x2018,x2019,x2020,x2021,x2022,x2023
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,405586592.178771,487709497.206704,596648044.692737,695530726.256983,764804469.273743,872067039.106145,958659217.877095,1083240223.463690,1245810055.865920,1320670391.061450,1379888268.156420,1531843575.418990,1665363128.491620,1722905027.932960,1873452513.966480,1896456983.240220,1961843575.418990,2044111731.843580,2254830726.256980,2360017318.435750,2469782681.564250,2677641340.782120,2843024581.005590,2553793296.089390,2453597206.703910,2637859217.877090,2615208379.888270,2727849720.670390,2790849720.670390,2962907262.569830,2983635195.530730,3092429050.279330,3276184357.541900,3395798882.681560,2558906303.880980,3103184101.513540,3544707788.056640,nan
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,21216962289.557598,22307471355.677898,23702472100.002300,25779376632.721802,28049537324.857399,30374910059.507801,33049155473.497601,35933757401.763100,38749864977.022102,42964345101.172501,43702318276.335297,47632993912.839798,51627511756.539497,66610917871.453697,81658052383.255707,86883552689.319504,88290755580.516296,98868135189.606903,110802408134.927002,129780974539.910995,168165164238.412994,177750099368.148010,169302766914.394989,179897833535.127014,166208098815.079010,143249519478.307007,154792610860.846985,185812458342.954987,202092611345.540985,214473497464.093994,251211844157.140015,273542601824.325012,239433170776.127991,240271389187.381012,243564813413.467010,273433056168.813995,272519342675.993011,288405283765.896973,268833638136.368011,265429291621.997986,287201651448.677979,260992227336.281006,267815037918.239990,355716369427.671021,442696185471.328979,516661066465.533997,580240749509.223999,665598701583.949951,713502105052.171021,715485327250.406006,849409658737.661011,945439147816.613037,952675592818.555054,963347276805.076050,979689662429.069946,899295676086.836060,829829959319.291992,940105480733.676025,1012719339457.719971,1006527294483.089966,929074086484.093018,1086772164579.869995,1183962133998.870117,1236163044999.969971
2,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,3521418059.923450,2813571753.872530,3825701438.999630,4520946818.545810,5224896718.677820,6203256538.709670,6971758282.293510,9747886187.393930,10109297047.543200,12416152732.056700,15856668555.833599,17805098206.314098,19907329777.587200,20146416757.598701,20497128555.697201,19134221644.732498,18116572395.077202,18753456497.815899,18053222687.412601,18799444490.112801,19955929052.149601,14266499429.874599,14502158192.090401,nan
3,Africa Western and Central,AFW,GDP (current US$),NY.GDP.MKTP.CD,11884128412.356400,12685662254.548300,13606829296.953501,14439975113.762899,15769107619.644300,16934480009.675100,18048039301.765301,16495768073.440901,17022696447.026100,19301308226.682098,26697107727.570499,24507115744.839901,29493915342.496899,36902672687.493500,49688193508.801102,57280847233.579002,68386320009.244102,71790695539.893600,78779099935.404999,96682239143.433395,120501079996.362000,216911025160.501007,195917937082.071991,151222762742.845001,131218159345.529999,137802915765.208008,109237001606.117004,112294774121.175003,111227154884.746994,103934371464.606995,123711320144.873993,129942000513.294006,124599388202.386993,129732548073.785004,134776896049.753006,207176985178.438995,262

,game,medal_type,country_name,game_location,game_season,game_year
0,athens-1896,GOLD,Australia,Greece,Summer,1896
1,athens-1896,GOLD,Australia,Greece,Summer,1896
2,athens-1896,BRONZE,Austria,Greece,Summer,1896
3,athens-1896,GOLD,Austria,Greece,Summer,1896
4,athens-1896,BRONZE,Austria,Greece,Summer,1896


,entity,code,year,population
0,Afghanistan,AFG,1950,7776182
1,Afghanistan,AFG,1951,7879343
2,Afghanistan,AFG,1952,7987783
3,Afghanistan,AFG,1953,8096703
4,Afghanistan,AFG,1954,8207953


### Q1- Transformations A. Melt

Het GDP-data.frame is in ‘messy format’. Ieder jaar heeft namelijk een eigen kolom,
waardoor een nieuw jaar toevoegen het toevoegen van een nieuwe kolom betekent. Het
is netter wanneer nieuwe observaties nieuwe rijen betekenen.
Maak vanuit het ruwe GDP-data.frame een nieuw data.frame met kolommen:
- `country_name`
- `country_code`
- `year` (het jaartal, afkomstig uit de kolomnamen in het format ‘xYYYY’). **Tip**: Gebruik value_vars=<your_dataframe>.columns[4:], in de functie om alle jaarkolommen op te halen. Hiermee worden alle kolommen vanaf positie 5 meegenomen.
- `gdp` (de waarde van gdp, afkomstig uit die kolommen)


Zie onderstaand voorbeeld hoe het getransformeerde GDP-data.frame eruit zou moeten zien.

![title](Voorbeelden/Q1_Transformaties_A_Voorbeeld.png)

In [147]:
import pycountry

def name_to_iso3(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except:
        return None

olympics_raw["country_code"] = olympics_raw["country_name"].apply(name_to_iso3)
print(olympics_raw.head(2))

flags_raw["country_code"] = flags_raw["name"].apply(name_to_iso3)
print(flags_raw.head(2))

          game medal_type country_name game_location game_season  game_year  \
0  athens-1896       GOLD    Australia        Greece      Summer       1896   
1  athens-1896       GOLD    Australia        Greece      Summer       1896   

  country_code  
0          AUS  
1          AUS  
          name  flag_nr_colors flag_mainhue flag_topleft_color  \
0  Afghanistan               5        green              black   
1      Albania               3          red                red   

  flag_botright_color country_code country_name_official  
0               green          AFG           Afghanistan  
1                 red          ALB               Albania  


In [148]:
# Specifieke kolommen hernoemen
population_raw = population_raw.rename(columns={
    'entity': 'country_name',
    'code': 'country_code'
})
print(population_raw.columns)   

Index(['country_name', 'country_code', 'year', 'population'], dtype='object')


In [146]:
# Verwijder rijen waar country_code leeg/missing is uit population_raw
population_clean = population_raw[population_raw['code'].notna() & (population_raw['code'] != '')]
print(f"Origineel: {len(population_raw)} rijen")
print(f"Na filtering: {len(population_clean)} rijen")
print(f"Verwijderd: {len(population_raw) - len(population_clean)} rijen")
print(population_clean.columns)

# Voor GDP
gdp_clean = gdp_raw[gdp_raw['country_code'].notna() & (gdp_raw['country_code'] != '')]
print(f"GDP origineel: {len(gdp_raw)} rijen")
print(f"GDP na filtering: {len(gdp_clean)} rijen")
print(f"GDP verwijderd: {len(gdp_raw) - len(gdp_clean)} rijen")
print(gdp_clean.columns)


# Voor Olympische Spelen
olympics_clean = olympics_raw[olympics_raw['country_code'].notna() & (olympics_raw['country_code'] != '')]
print(f"Olympics origineel: {len(olympics_raw)} rijen")
print(f"Olympics na filtering: {len(olympics_clean)} rijen")
print(f"Olympics verwijderd: {len(olympics_raw) - len(olympics_clean)} rijen")
print(olympics_clean.columns)





Origineel: 18944 rijen
Na filtering: 17612 rijen
Verwijderd: 1332 rijen
Index(['entity', 'code', 'year', 'population'], dtype='object')
GDP origineel: 266 rijen
GDP na filtering: 266 rijen
GDP verwijderd: 0 rijen
Index(['country_name', 'country_code', 'indicator_name', 'indicator_code',
       'x1960', 'x1961', 'x1962', 'x1963', 'x1964', 'x1965', 'x1966', 'x1967',
       'x1968', 'x1969', 'x1970', 'x1971', 'x1972', 'x1973', 'x1974', 'x1975',
       'x1976', 'x1977', 'x1978', 'x1979', 'x1980', 'x1981', 'x1982', 'x1983',
       'x1984', 'x1985', 'x1986', 'x1987', 'x1988', 'x1989', 'x1990', 'x1991',
       'x1992', 'x1993', 'x1994', 'x1995', 'x1996', 'x1997', 'x1998', 'x1999',
       'x2000', 'x2001', 'x2002', 'x2003', 'x2004', 'x2005', 'x2006', 'x2007',
       'x2008', 'x2009', 'x2010', 'x2011', 'x2012', 'x2013', 'x2014', 'x2015',
       'x2016', 'x2017', 'x2018', 'x2019', 'x2020', 'x2021', 'x2022', 'x2023'],
      dtype='object')
Olympics origineel: 20170 rijen
Olympics na filtering: 17

In [ ]:

# After melting, remove the 'x' prefix from the year column and convert to integer. This is necessary for correct merging later.
<your_dataframe> = <your_dataframe>.assign(year=<your_dataframe>['year']
                                           .str.replace('x', '')
                                           .astype(int))

In [ ]:
# Smelten van alle jaar-kolommen (vanaf kolom 4, index = 4) uit het originele gdp_raw dataframe
gdp_tidy = gdp_clean.melt(
    id_vars=["country_name", "country_code"],          # kolommen die blijven
    value_vars=gdp_clean.columns[4:],                   # alle jaar-kolommen
    var_name="year",                                  # nieuwe kolomnaam voor jaarkolommen
    value_name="gdp"                                  # nieuwe kolomnaam voor waarden
)

# Verwijder de 'x' prefix uit de jaar-kolommen en zet om naar int
gdp_tidy["year"] = gdp_tidy["year"].str.replace("x", "").astype(int)

# Controleer resultaat
print("Transformatie voltooid — shape:", gdp_tidy.shape)
print("Transformatie voltooid — columns:", gdp_tidy.columns)
display(gdp_tidy.head(10))



In [143]:
# Smelten van alle jaar-kolommen (vanaf kolom 4, index = 4) uit het originele gdp_raw dataframe
gdp_tidy = gdp_clean.melt(
    id_vars=["country_name", "country_code"],          # kolommen die blijven
    value_vars=gdp_clean.columns[4:],                   # alle jaar-kolommen
    var_name="year",                                  # nieuwe kolomnaam voor jaarkolommen
    value_name="gdp"                                  # nieuwe kolomnaam voor waarden
)

# Verwijder de 'x' prefix uit de jaar-kolommen en zet om naar int
gdp_tidy["year"] = gdp_tidy["year"].str.replace("x", "").astype(int)

# Controleer resultaat
print("Transformatie voltooid — shape:", gdp_tidy.shape)
print("Transformatie voltooid — columns:", gdp_tidy.columns)
display(gdp_tidy.head(10))


Transformatie voltooid — shape: (17024, 4)
Transformatie voltooid — columns: Index(['country_name', 'country_code', 'year', 'gdp'], dtype='object')


,country_name,country_code,year,gdp
0,Aruba,ABW,1960,NaN
1,Africa Eastern and Southern,AFE,1960,2.121696e+10
2,Afghanistan,AFG,1960,NaN
3,Africa Western and Central,AFW,1960,1.188413e+10
4,Angola,AGO,1960,NaN
5,Albania,ALB,1960,NaN
6,Andorra,AND,1960,NaN
7,Arab World,ARB,1960,NaN
8,United Arab Emirates,ARE,1960,NaN
9,Argentina,ARG,1960,NaN


In [137]:
# Methode 1: drop_duplicates (meest gebruikt)
gdp_distinct = gdp_tidy[['country_name', 'country_code']].drop_duplicates()

# Methode 2: met reset_index voor schone index
gdp_distinct = gdp_tidy[['country_name', 'country_code']].drop_duplicates().reset_index(drop=True)

print(f"Aantal unieke landen: {len(gdp_distinct)}")
display(gdp_distinct.head())

# Rijen waar country_code leeg is (NaN, None, lege string)
gdp_empty_codes = gdp_tidy[gdp_tidy['country_code'].isna() | (gdp_tidy['country_code'] == '')]

print(f"Aantal rijen met lege country_code: {len(gdp_empty_codes)}")
display(gdp_empty_codes.head())

# Alternatief: alleen controleren op NaN/None
gdp_missing_codes = gdp_tidy[gdp_tidy['country_code'].isna()]
print(f"Aantal rijen met missing country_code: {len(gdp_missing_codes)}")

Aantal unieke landen: 266


,country_name,country_code
0,Aruba,ABW
1,Africa Eastern and Southern,AFE
2,Afghanistan,AFG
3,Africa Western and Central,AFW
4,Angola,AGO


Aantal rijen met lege country_code: 0


,country_name,country_code,year,gdp


Aantal rijen met missing country_code: 0


### Q1- Transformations B. Pivot

Ook het olympics-data.frame is niet in het format die we nodig hebben voor onze verder
analyse. Iedere regel is namelijk een enkele medaille, terwijl wij het aantal medailles per
land willen weten. Vind, per Land / Olympische Spelen / Medaillekleur-combinatie het
aantal medailles. Zorg dat je niet de informatie over locatie, seizoen en jaar verliest.


Vervolgens willen we graag dat GOLD, SILVER en BRONZE ieder een eigen kolom
hebben met het aantal medailles behaald van die kleur. Zie onderstaand voorbeeld hoe
het getransformeerde olympics-data.frame eruit zou moeten zien.

**Tip** Gebruik de argumenten aggfunc='size' (om medailles kleuren te tellen.) , fill_value=0 (voorkomt NaN, bij landen zonder die medaille kleur)

![title](Voorbeelden/Q1_Transformaties_B_Voorbeeld.png)

In [17]:
# Use pd.pivot_table instead of pd.pivot to handle the extra arguments aggfunc/fill_value. pivot_table is a more flexible function with the same syntax.

# After pivoting the medal_type to columns, reset the index to turn the multi-index into columns for easier merging later.
#<your_dataframe>.reset_index(inplace=True) 
#<your_dataframe>.columns.name = None  # Remove the 'medal_type' name from columns - not needed but looks better

#Pivot: Aantal medailles per Land / Jaar / Seizoen / Locatie ===

# 1 Alleen rijen met een medaille behouden
df = df[df["medal_type"].notna()].copy()

# 2 Kolommen netjes maken (consistent hoofdletters, etc.)
df["medal_type"] = df["medal_type"].str.upper().str.strip()

# 3️ Maak de pivot-tabel:
#    - Index: unieke combinaties van land, spelen, jaar, seizoen, stad
#    - Columns: medaille-kleur (GOLD/SILVER/BRONZE)
#    - Values: telling (aggfunc='size')
#    - fill_value=0 zorgt dat landen zonder bepaalde kleur gewoon 0 krijgen
olympics_pivot = pd.pivot_table(
    df,
    index=["country_name", "game", "game_year", "game_season", "game_location"],
    columns="medal_type",
    aggfunc="size",
    fill_value=0
)

# 4️ Index terug naar kolommen (multi-index → platte tabel)
olympics_pivot.reset_index(inplace=True)

# 5️ Optioneel: kolomtitel "medal_type" weghalen (ziet er netter uit)
olympics_pivot.columns.name = None

# 6️ Controleer resultaat
print("Pivot voltooid — shape:", olympics_pivot.shape)
print("Pivot voltooid — columns:", olympics_pivot.columns)
display(olympics_pivot.head(10).style.set_caption("Aantal medailles per Land/Jaar/Seizoen"))


Pivot voltooid — shape: (1779, 8)
Pivot voltooid — columns: Index(['country_name', 'game', 'game_year', 'game_season', 'game_location',
       'BRONZE', 'GOLD', 'SILVER'],
      dtype='object')


,country_name,game,game_year,game_season,game_location,BRONZE,GOLD,SILVER
0,Afghanistan,beijing-2008,2008,Summer,China,1,0,0
1,Afghanistan,london-2012,2012,Summer,Great Britain,1,0,0
2,Algeria,atlanta-1996,1996,Summer,United States,1,2,0
3,Algeria,barcelona-1992,1992,Summer,Spain,1,1,0
4,Algeria,beijing-2008,2008,Summer,China,1,0,1
5,Algeria,london-2012,2012,Summer,Great Britain,0,1,0
6,Algeria,los-angeles-1984,1984,Summer,United States,2,0,0
7,Algeria,rio-2016,2016,Summer,Brazil,0,0,2
8,Algeria,sydney-2000,2000,Summer,Australia,3,1,1
9,Argentina,amsterdam-1928,1928,Summer,Netherlands,1,3,3


## Q2 – Joins
Voeg de dataframes (olympics, gdp, population, flags) samen tot één dataframe, alle observaties uit olympics behoudend. Gebruik de juiste keys.

In [102]:
#Loaded: flags_raw                 shape=(193, 5)  from 'flags_raw.csv'
#Loaded: gdp_raw                   shape=(266, 68)  from 'gdp_raw.csv'
#Loaded: olympics_raw              shape=(20170, 6)  from 'olympics_raw.csv'
#Loaded: population_raw            shape=(18944, 4)  from 'population_raw.csv'

print('----Flags------------------------------')
#print("Pivot voltooid — Flags columns:", flags_raw.columns)
print(flags_raw.head(2))
print('------GDP----------------------------')
#print("Transformatie voltooid — GDP columns:", gdp_tidy.columns)
print(gdp_tidy.head(2))
print('-----Olympics-----------------------------')    
#print("Pivot voltooid — Olympics columns:", olympics_pivot.columns)
print(olympics_pivot.head(2))
print('---Population-------------------------------')
#print("Pivot voltooid — Population columns:", population_raw.columns)
print(population_raw.head(2))
print('----------------------------------')

----Flags------------------------------
          name  flag_nr_colors flag_mainhue flag_topleft_color  \
0  Afghanistan               5        green              black   
1      Albania               3          red                red   

  flag_botright_color  
0               green  
1                 red  
------GDP----------------------------
                  country_name country_code  year           gdp
0                        Aruba          ABW  1960           NaN
1  Africa Eastern and Southern          AFE  1960  2.121696e+10
-----Olympics-----------------------------
  country_name          game  game_year game_season  game_location  BRONZE  \
0  Afghanistan  beijing-2008       2008      Summer          China       1   
1  Afghanistan   london-2012       2012      Summer  Great Britain       1   

   GOLD  SILVER  
0     0       0  
1     0       0  
---Population-------------------------------
        entity code  year  population
0  Afghanistan  AFG  1950     7776182
1  Afg

In [103]:
# Population tidy: alleen kolomnamen aanpassen zodat ze overeenkomen met andere dataframes
population_tidy = (
    population_raw
    .rename(columns={"entity": "country_name", "code": "country_code"})
    [["country_name", "country_code", "year", "population"]]
    .copy()
)
print(population_tidy.head(2))

  country_name country_code  year  population
0  Afghanistan          AFG  1950     7776182
1  Afghanistan          AFG  1951     7879343


In [110]:
# Flags tidy: alleen kolomnamen aanpassen zodat ze overeenkomen met andere dataframes
flags_tidy = (
    flags_raw
    .rename(columns={"name": "country_name"})
    [["country_name", "flag_nr_colors", "flag_mainhue", "flag_topleft_color", "flag_botright_color"]]
    .copy()
)
print(flags_tidy.head(2))

  country_name  flag_nr_colors flag_mainhue flag_topleft_color  \
0  Afghanistan               5        green              black   
1      Albania               3          red                red   

  flag_botright_color  
0               green  
1                 red  


In [111]:
# --- 1) Normaliseer sleutelkolommen
# zet alles om naar hoofdletters, zodat "usa", "Usa", "USA " → allemaal "USA" worden
def Normaliseer(df, cc="country_code"):
    if cc in df.columns:
        df[cc] = df[cc].astype(str).str.strip().str.upper()
    return df

def ZetOmNaarNumeriek_Year(df, y="year"):
    if y in df.columns and not pd.api.types.is_integer_dtype(df[y]):
        # forceer naar int waar mogelijk
        df[y] = pd.to_numeric(df[y], errors="coerce").astype("Int64")
        # als er NA's in year zitten, laat ze als NA; merges behouden dan geen match
        # optioneel: rijen met NA-year verwijderen:
        # df = df.dropna(subset=[y])
    return df

olym = Normaliseer(olympics_pivot.copy())
olym = ZetOmNaarNumeriek_Year(olym)

gdp  = Normaliseer(gdp_tidy.copy())
gdp  = ZetOmNaarNumeriek_Year(gdp)

pop  = Normaliseer(population_tidy.copy())
pop  = ZetOmNaarNumeriek_Year(pop)

flags = Normaliseer(flags_raw.copy())
print(flags_raw.head(2))
print(flags.head(2))


          name  flag_nr_colors flag_mainhue flag_topleft_color  \
0  Afghanistan               5        green              black   
1      Albania               3          red                red   

  flag_botright_color  
0               green  
1                 red  
          name  flag_nr_colors flag_mainhue flag_topleft_color  \
0  Afghanistan               5        green              black   
1      Albania               3          red                red   

  flag_botright_color  
0               green  
1                 red  


             name country_code country_name_official
0     Afghanistan          AFG           Afghanistan
1         Albania          ALB               Albania
2         Algeria          DZA               Algeria
3  American-Samoa         None                  None
4         Andorra          AND               Andorra


In [115]:
import pandas as pd
from typing import List, Tuple, Optional

def Ontdubbel_first_with_dupes(df: pd.DataFrame,
                               keys: List[str],
                               value_cols: List[str]) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Ontdubbel op `keys` (behoud eerste niet-NA per value_col) en geef ook alle dubbele records terug.
    """
    missing = [k for k in keys if k not in df.columns]
    if missing:
        raise KeyError(f"Ontbrekende sleutelkolommen: {missing}")

    keep_cols = [c for c in (list(keys) + list(value_cols)) if c in df.columns]
    d = df[keep_cols].copy()

    # markeer alle rijen die onderdeel zijn van een dubbele sleutel
    mask_dupes = d.duplicated(subset=keys, keep=False)
    duplicates = df.loc[mask_dupes].sort_values(keys).copy()  # volledige rijen voor rapport

    # stabiele selectie
    d_sorted = d.sort_values(keys)

    # 'first' pakt de eerste (incl. eerste niet-NA voor pandas>=2.0 op series?)—veilig voor Int64/float
    agg_map = {c: "first" for c in value_cols if c in d_sorted.columns}
    deduped = d_sorted.groupby(keys, as_index=False).agg(agg_map)

    return deduped, duplicates


def dq_duplicates_report(df: pd.DataFrame,
                         keys: List[str],
                         value_cols: List[str],
                         excel_path: Optional[str] = None,
                         sheet_prefix: str = "DQ") -> dict:
    """
    Maakt een DQ-rapport over duplicaten en (optioneel) een Excel-bestand.

    Returns dict met:
      - 'deduped': ontdubbelde DataFrame
      - 'duplicates': alle dubbele rijen (volledig)
      - 'counts_by_key': aantallen per sleutelcombinatie (>1 = duplicaat)
      - 'summary': 1-rijig DataFrame met kerncijfers
    """
    deduped, duplicates = Ontdubbel_first_with_dupes(df, keys, value_cols)

    # aantallen per sleutelcombinatie
    key_counts = df.groupby(keys, dropna=False, as_index=False).size().rename(columns={"size": "count"})
    counts_by_key = key_counts.sort_values(["count"] + keys, ascending=[False] + [True]*len(keys))

    total_rows = len(df)
    total_unique_keys = (counts_by_key["count"] >= 1).sum()
    duplicate_groups = (counts_by_key["count"] > 1).sum()
    duplicate_rows = int((counts_by_key["count"] - 1).clip(lower=0).sum())  # extra rijen boven de eerste

    summary = pd.DataFrame([{
        "total_rows": total_rows,
        "total_unique_keys": total_unique_keys,
        "duplicate_key_groups": duplicate_groups,
        "duplicate_rows": duplicate_rows,
        "pct_rows_duplicate": round(100 * duplicate_rows / total_rows, 2) if total_rows else 0.0
    }])

    if excel_path:
        with pd.ExcelWriter(excel_path, engine="xlsxwriter") as xw:
            summary.to_excel(xw, index=False, sheet_name=f"{sheet_prefix}_Summary")
            duplicates.to_excel(xw, index=False, sheet_name=f"{sheet_prefix}_DuplicateRows")
            counts_by_key.to_excel(xw, index=False, sheet_name=f"{sheet_prefix}_CountsByKey")
            # simpele autofit (optioneel, kan leeg blijven of worden uitgewerkt)
            for sheet in [f"{sheet_prefix}_Summary",
                          f"{sheet_prefix}_DuplicateRows",
                          f"{sheet_prefix}_CountsByKey"]:
                ws = xw.sheets[sheet]
                # Autofit kan hier worden toegevoegd indien gewenst
    return {
        "deduped": deduped,
        "duplicates": duplicates,
        "counts_by_key": counts_by_key,
        "summary": summary
    }

res_gdp = dq_duplicates_report(
    df=gdp_tidy,
    keys=["country_code", "year"],
    value_cols=["gdp"],
    excel_path=None  # of bijv. "dq_report_gdp.xlsx" om Excel te schrijven
)

gdp_clean      = res_gdp["deduped"]
gdp_dupes      = res_gdp["duplicates"]
gdp_counts_key = res_gdp["counts_by_key"]
gdp_summary    = res_gdp["summary"]

res_pop = dq_duplicates_report(
    df=population_tidy,
    keys=["country_code", "year"],
    value_cols=["population"],
    excel_path=None  # of "dq_report_population.xlsx"
)


pop_clean      = res_pop["deduped"]
pop_dupes      = res_pop["duplicates"]
pop_counts_key = res_pop["counts_by_key"]
pop_summary    = res_pop["summary"]


res_flags = dq_duplicates_report(
    df=flags_tidy,
    keys=["country_name"],  # unieke sleutel
    value_cols=["flag_nr_colors", "flag_mainhue", "flag_topleft_color", "flag_botright_color"],
    excel_path=None  # of bijv. "dq_report_flags.xlsx"
)

flags_clean      = res_flags["deduped"]
flags_dupes      = res_flags["duplicates"]
flags_counts_key = res_flags["counts_by_key"]
flags_summary    = res_flags["summary"]




print("GDP – samenvatting:\n", gdp_summary.to_string(index=False))
print("\nTop 10 sleutelcombinaties met meeste duplicaten (GDP):")
print(gdp_counts_key[gdp_counts_key["count"] > 1].head(10).to_string(index=False))

print("\nPopulation – samenvatting:\n", pop_summary.to_string(index=False))
print("\nTop 10 sleutelcombinaties met meeste duplicaten (Population):")
print(pop_counts_key[pop_counts_key["count"] > 1].head(10).to_string(index=False))

print("Flags – samenvatting:\n", flags_summary.to_string(index=False))
print("\nTop 10 sleutelcombinaties met meeste duplicaten (Flags):")
print(flags_counts_key[flags_counts_key["count"] > 1].head(10).to_string(index=False))


GDP – samenvatting:
  total_rows  total_unique_keys  duplicate_key_groups  duplicate_rows  pct_rows_duplicate
      17024              17024                     0               0                 0.0

Top 10 sleutelcombinaties met meeste duplicaten (GDP):
Empty DataFrame
Columns: [country_code, year, count]
Index: []

Population – samenvatting:
  total_rows  total_unique_keys  duplicate_key_groups  duplicate_rows  pct_rows_duplicate
      18944              17686                    74            1258                6.64

Top 10 sleutelcombinaties met meeste duplicaten (Population):
country_code  year  count
         NaN  1950     18
         NaN  1951     18
         NaN  1952     18
         NaN  1953     18
         NaN  1954     18
         NaN  1955     18
         NaN  1956     18
         NaN  1957     18
         NaN  1958     18
         NaN  1959     18
Flags – samenvatting:
  total_rows  total_unique_keys  duplicate_key_groups  duplicate_rows  pct_rows_duplicate
        193   

GDP – samenvatting:
  total_rows  total_unique_keys  duplicate_key_groups  duplicate_rows  pct_rows_duplicate
      17024              17024                     0               0                 0.0

Top 10 sleutelcombinaties met meeste duplicaten (GDP):
Empty DataFrame
Columns: [country_code, year, count]
Index: []

Population – samenvatting:
  total_rows  total_unique_keys  duplicate_key_groups  duplicate_rows  pct_rows_duplicate
      18944              17686                    74            1258                6.64

Top 10 sleutelcombinaties met meeste duplicaten (Population):
country_code  year  count
         NaN  1950     18
         NaN  1951     18
         NaN  1952     18
         NaN  1953     18
         NaN  1954     18
         NaN  1955     18
         NaN  1956     18
         NaN  1957     18
         NaN  1958     18
         NaN  1959     18


In [ ]:
# Left joins uitvoeren (alle olympics-observaties behouden)
merged = (
    olym
    .merge(gdp, how="left", on=["country_code", "year"], validate="m:1")
    .merge(pop_, how="left", on=["country_code", "year"], validate="m:1")
    .merge(flags_tidy.drop_duplicates(subset=["country_code"]), how="left", on="country_code", validate="m:1")
)

KeyError: 'country_code'

In [ ]:


# --- 4) Kolommen logisch ordenen: metadata → medailles → indicatoren
meta_cols = [c for c in ["country_name","country_code","games","year","season","city"] if c in merged.columns]
medal_cols = [c for c in ["GOLD","SILVER","BRONZE"] if c in merged.columns]
indicator_cols = [c for c in ["gdp","population"] if c in merged.columns]
other_cols = [c for c in merged.columns if c not in (meta_cols + medal_cols + indicator_cols)]

merged = merged[meta_cols + medal_cols + indicator_cols + other_cols]

# --- 5) Output check
merged.index = merged.index + 1
print("✅ Q2 Joins voltooid — vorm:", merged.shape)
print("  - Aantal unieke (country_code, year) in olympics:", olymp[["country_code","year"]].drop_duplicates().shape[0])
print("  - % matches met GDP:", round(100 * merged["gdp"].notna().mean(), 1), "%")
print("  - % matches met Population:", round(100 * merged["population"].notna().mean(), 1), "%")

display(merged.head(12).style.set_caption("Olympics × GDP × Population × Flags (left join op olympics)"))

## Q3 – Functions and Conditions
Maak een functie met als output: een lijst van de landen die bij alle Spelen van een bepaald seizoen meer dan `x` gouden medailles hebben gehaald. Aggregeer en sommeer op medaillekleur per land.

De input argumenten van deze functie zijn:
- de data (het dataframe uit Q2)
- het seizoen (`Winter` of `Summer`)
- het aantal medailles (de grens waarboven het land in de output moet
voorkomen)

Zorg dat de functie ook werkt wanneer alleen de data wordt ingegeven (de andere twee
input argumenten moeten dus default-waardes hebben).

Zorg dat de functie een informatieve warning geeft wanneer input argument seizoen
geen valide waarde bevat.

## Q4 – Functions and Iteration
Update de functie in Q3, door de volgende input argumenten toe te voegen:
- `kleur` (GOLD, SILVER, BRONZE)
- een parameter om te kiezen of je landen wilt met méér of minder dan de opgegeven grenswaarde als output.

Gebruik de functie om de volgende lijst als output te krijgen:
- Alle landen die tijdens alle Zomerspelen in totaal minder dan 1 gouden medaille hebben gehaald.


Itereer over de functie om het volgende te vinden:
- Alle landen die tijdens alle zomerspelen in totaal meer dan 50, meer dan 100, meer dan 250 en meer dan 500 gouden medailles hebben gehaald.


Dit zijn dus 4 lijstjes. Gebruik loops.